In [1]:
import pandas as pd
import numpy as np
import os
from statsmodels.formula.api import logit
import matplotlib.pyplot as plt

# Define a class to mimic trial_sequence
class TrialSequence:
    def __init__(self, estimand):
        self.estimand = estimand
        self.data = None
        self.switch_weights = {}
        self.censor_weights = {}
        self.outcome_model = None
        self.expansion = None

    def set_data(self, data, id_col, period_col, treatment_col, outcome_col, eligible_col):
        self.data = data[[id_col, period_col, treatment_col, outcome_col, eligible_col]].copy()
        self.data.columns = ['id', 'period', 'treatment', 'outcome', 'eligible']
        return self

# Create directories
trial_pp_dir = "trial_pp"
trial_itt_dir = "trial_itt"
os.makedirs(trial_pp_dir, exist_ok=True)
os.makedirs(trial_itt_dir, exist_ok=True)

# Initialize trial objects
trial_pp = TrialSequence(estimand="PP")
trial_itt = TrialSequence(estimand="ITT")

---

In [2]:
# Load data
data_censored = pd.read_csv("data_censored.csv")

# Set data for PP
trial_pp = trial_pp.set_data(
    data=data_censored,
    id_col="id",
    period_col="period",
    treatment_col="treatment",
    outcome_col="outcome",
    eligible_col="eligible"
)

# Set data for ITT
trial_itt = trial_itt.set_data(
    data=data_censored,
    id_col="id",
    period_col="period",
    treatment_col="treatment",
    outcome_col="outcome",
    eligible_col="eligible"
)

# Display head of data to match R output
print(data_censored.head(6))

   id  period  treatment  x1        x2  x3        x4  age     age_s  outcome  \
0   1       0          1   1  1.146148   0  0.734203   36  0.083333        0   
1   1       1          1   1  0.002200   0  0.734203   37  0.166667        0   
2   1       2          1   0 -0.481762   0  0.734203   38  0.250000        0   
3   1       3          1   0  0.007872   0  0.734203   39  0.333333        0   
4   1       4          1   1  0.216054   0  0.734203   40  0.416667        0   
5   1       5          1   0 -0.057482   0  0.734203   41  0.500000        0   

   censored  eligible  
0         0         1  
1         0         0  
2         0         0  
3         0         0  
4         0         0  
5         1         0  


---

In [5]:
def set_switch_weight_model(trial, numerator, denominator, save_path):
    # Store the settings in the trial object
    trial.switch_weights = {
        'numerator_formula': numerator,
        'denominator_formula': denominator,
        'model_fitter': 'te_stats_glm_logit',  # Match R's model fitter type
        'save_path': save_path,
        'fitted': False  # Indicate models are not yet fitted
    }
    
    # Create a formatted string to match the R output
    output = (
        "  - Numerator formula: {}\n"
        "  - Denominator formula: {}\n"
        "  - Model fitter type: te_stats_glm_logit\n"
        "  - Weight models not fitted. Use calculate_weights()"
    ).format(numerator, denominator)
    
    return trial, output

# Apply to trial_pp
trial_pp, switch_weights_output = set_switch_weight_model(
    trial_pp,
    numerator="treatment ~ age",
    denominator="treatment ~ age + x1 + x3",
    save_path=os.path.join(trial_pp_dir, "switch_models")
)

# Print the formatted output
print("Switch Weights for PP:")
print(switch_weights_output)

Switch Weights for PP:
  - Numerator formula: treatment ~ age
  - Denominator formula: treatment ~ age + x1 + x3
  - Model fitter type: te_stats_glm_logit
  - Weight models not fitted. Use calculate_weights()


In [6]:
# Define the function (shared for both, but called separately)
def set_censor_weight_model(trial, censor_event, numerator, denominator, pool_models, save_path):
    trial.censor_weights = {
        'censor_event': censor_event,
        'numerator_formula': f"1 - {censor_event} ~ {numerator}",
        'denominator_formula': f"1 - {censor_event} ~ {denominator}",
        'pool_models': pool_models,
        'model_fitter': 'te_stats_glm_logit',
        'save_path': save_path,
        'fitted': False
    }
    
    output = (
        f"Censor Weights for {trial.estimand}:\n"
        f"  - Numerator formula: 1 - {censor_event} ~ {numerator}\n"
        f"  - Denominator formula: 1 - {censor_event} ~ {denominator}\n"
    )
    
    if pool_models == "numerator":
        output += (
            "  - Numerator model is pooled across treatment arms. Denominator model is not pooled.\n"
        )
    
    output += (
        "  - Model fitter type: te_stats_glm_logit\n"
        "  - Weight models not fitted. Use calculate_weights()"
    )
    
    return trial, output

# Apply to trial_pp
trial_pp, pp_censor_weights_output = set_censor_weight_model(
    trial_pp,
    censor_event="censored",
    numerator="x2",
    denominator="x2 + x1",
    pool_models="none",
    save_path=os.path.join(trial_pp_dir, "switch_models")
)

# Print trial_pp output
print(pp_censor_weights_output)


Censor Weights for PP:
  - Numerator formula: 1 - censored ~ x2
  - Denominator formula: 1 - censored ~ x2 + x1
  - Model fitter type: te_stats_glm_logit
  - Weight models not fitted. Use calculate_weights()


In [7]:
# Apply to trial_itt (using the same function defined above)
trial_itt, itt_censor_weights_output = set_censor_weight_model(
    trial_itt,
    censor_event="censored",
    numerator="x2",
    denominator="x2 + x1",
    pool_models="numerator",
    save_path=os.path.join(trial_itt_dir, "switch_models")
)

# Print trial_itt output
print(itt_censor_weights_output)

Censor Weights for ITT:
  - Numerator formula: 1 - censored ~ x2
  - Denominator formula: 1 - censored ~ x2 + x1
  - Numerator model is pooled across treatment arms. Denominator model is not pooled.
  - Model fitter type: te_stats_glm_logit
  - Weight models not fitted. Use calculate_weights()



---

In [8]:
import os
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Step 1-3: Setup
class TrialSequence:
    def __init__(self, estimand):
        self.estimand = estimand
        self.data = None
        self.censor_weights = {}
        self.switch_weights = {}

    def set_data(self, data, id_col, period_col, treatment_col, outcome_col, eligible_col, censor_col="censored"):
        self.data = data[[id_col, period_col, treatment_col, outcome_col, eligible_col, censor_col]].copy()
        self.data.columns = ['id', 'period', 'treatment', 'outcome', 'eligible', 'censored']
        return self

def set_censor_weight_model(trial, censor_event, numerator, denominator, pool_models, save_path):
    trial.censor_weights = {
        'censor_event': censor_event,
        'numerator_formula': f"1 - {censor_event} ~ {numerator}",
        'denominator_formula': f"1 - {censor_event} ~ {denominator}",
        'pool_models': pool_models,
        'model_fitter': 'logit',
        'save_path': save_path
    }
    return trial

trial_itt_dir = "trial_itt"
os.makedirs(trial_itt_dir, exist_ok=True)
trial_itt_subdir = os.path.join(trial_itt_dir, "switch_models")
os.makedirs(trial_itt_subdir, exist_ok=True)

trial_itt = TrialSequence(estimand="ITT")
data_censored = pd.read_csv("data_censored.csv")
trial_itt = trial_itt.set_data(
    data=data_censored,
    id_col="id",
    period_col="period",
    treatment_col="treatment",
    outcome_col="outcome",
    eligible_col="eligible",
    censor_col="censored"
)
trial_itt = set_censor_weight_model(
    trial_itt,
    censor_event="censored",
    numerator="x2",
    denominator="x2 + x1",
    pool_models="numerator",
    save_path=trial_itt_subdir
)

# Step 4: Calculate Weights with Debugging
def calculate_weights(trial):
    data = trial.data.merge(data_censored[['id', 'period', 'x1', 'x2']], on=['id', 'period'])
    os.makedirs(trial.censor_weights['save_path'], exist_ok=True)
    
    # Add lagged treatment, exclude period 0 for denominator
    data['prev_treatment'] = data.groupby('id')['treatment'].shift(1)
    
    # Numerator model (all data)
    data['not_censored'] = 1 - data['censored']
    print(f"Total observations for numerator: {len(data)}")
    num_model = logit("not_censored ~ x2", data=data).fit(disp=0)
    trial.censor_weights['n'] = num_model
    num_model.save(os.path.join(trial.censor_weights['save_path'], "model_n.pkl"))
    
    # Denominator models (exclude period 0)
    data_den = data[data['period'] > 0].dropna(subset=['not_censored', 'x2', 'x1', 'prev_treatment'])
    for prev_trt in [0, 1]:
        subset = data_den[data_den['prev_treatment'] == prev_trt]
        print(f"Observations for prev_treatment = {prev_trt}: {len(subset)}")
        den_model = logit("not_censored ~ x2 + x1", data=subset).fit(disp=0)
        trial.censor_weights[f'd{prev_trt}'] = den_model
        den_model.save(os.path.join(trial.censor_weights['save_path'], f"model_d{prev_trt}.pkl"))
    
    data['wt'] = 1.0
    trial.data = data
    print(f"Models stored: {list(trial.censor_weights.keys())}")
    return trial

trial_itt = calculate_weights(trial_itt)

# Show weight models
def show_weight_models(trial):
    print("Weight Models for Informative Censoring")
    print("---------------------------------------")
    
    # Numerator model
    if 'n' in trial.censor_weights:
        print("")
        print("[[n]]")
        print("Model: P(censor_event = 0 | X) for numerator")
        print("")
        print(" term        estimate   std.error statistic p.value")
        params = trial.censor_weights['n'].params
        std_err = trial.censor_weights['n'].bse
        z_stats = trial.censor_weights['n'].tvalues
        p_vals = trial.censor_weights['n'].pvalues
        for term in params.index:
            term_display = "(Intercept)" if term == "Intercept" else term
            print(f" {term_display:<12} {params[term]:>9.7f} {std_err[term]:>9.7f} {z_stats[term]:>9.7f} {p_vals[term]:.6e}")
        print("")
        print(f" null.deviance df.null logLik    AIC      BIC      deviance df.residual nobs")
        print(f" {trial.censor_weights['n'].llnull * -2:>9.4f}      {trial.censor_weights['n'].df_resid + trial.censor_weights['n'].df_model:<5.0f}     {trial.censor_weights['n'].llf:>9.4f} {trial.censor_weights['n'].aic:>7.4f} {trial.censor_weights['n'].bic:>7.4f} {trial.censor_weights['n'].llf * -2:>7.4f} {trial.censor_weights['n'].df_resid:<5.0f}         {trial.censor_weights['n'].nobs:<5.0f}")
        print("")
        print(f" path")
        print(f" {os.path.join(trial.censor_weights['save_path'], 'model_n.pkl')}")
    
    # Denominator models
    for key in ['d0', 'd1']:
        if key in trial.censor_weights:
            prev_trt = int(key[-1])
            print("")
            print(f"[[{key}]]")
            print(f"Model: P(censor_event = 0 | X, previous treatment = {prev_trt}) for denominator")
            print("")
            print(" term        estimate   std.error statistic p.value")
            params = trial.censor_weights[key].params
            std_err = trial.censor_weights[key].bse
            z_stats = trial.censor_weights[key].tvalues
            p_vals = trial.censor_weights[key].pvalues
            for term in params.index:
                term_display = "(Intercept)" if term == "Intercept" else term
                print(f" {term_display:<12} {params[term]:>9.7f} {std_err[term]:>9.7f} {z_stats[term]:>9.7f} {p_vals[term]:.6e}")
            print("")
            print(f" null.deviance df.null logLik    AIC      BIC      deviance df.residual nobs")
            print(f" {trial.censor_weights[key].llnull * -2:>9.4f}      {trial.censor_weights[key].df_resid + trial.censor_weights[key].df_model:<5.0f}     {trial.censor_weights[key].llf:>9.4f} {trial.censor_weights[key].aic:>7.4f} {trial.censor_weights[key].bic:>7.4f} {trial.censor_weights[key].llf * -2:>7.4f} {trial.censor_weights[key].df_resid:<5.0f}         {trial.censor_weights[key].nobs:<5.0f}")
            print("")
            print(f" path")
            print(f" {os.path.join(trial.censor_weights['save_path'], f'model_d{prev_trt}.pkl')}")

show_weight_models(trial_itt)

Total observations for numerator: 725
Observations for prev_treatment = 0: 337
Observations for prev_treatment = 1: 299
Models stored: ['censor_event', 'numerator_formula', 'denominator_formula', 'pool_models', 'model_fitter', 'save_path', 'n', 'd0', 'd1']
Weight Models for Informative Censoring
---------------------------------------

[[n]]
Model: P(censor_event = 0 | X) for numerator

 term        estimate   std.error statistic p.value
 (Intercept)  2.4480907 0.1405747 17.4148764 6.362614e-68
 x2           -0.4486482 0.1368779 -3.2777243 1.046476e-03

 null.deviance df.null logLik    AIC      BIC      deviance df.residual nobs
  404.2156      724       -196.7002 397.4004 406.5727 393.4004 723           725  

 path
 trial_itt\switch_models\model_n.pkl

[[d0]]
Model: P(censor_event = 0 | X, previous treatment = 0) for denominator

 term        estimate   std.error statistic p.value
 (Intercept)  2.7538302 0.3391049 8.1208807 4.628127e-16
 x2           -0.6289932 0.2524695 -2.4913629 1

In [9]:
import os
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Setup Classes and Functions
class TrialSequence:
    def __init__(self, estimand):
        self.estimand = estimand
        self.data = None
        self.censor_weights = {}
        self.switch_weights = {}

    def set_data(self, data, id_col, period_col, treatment_col, outcome_col, eligible_col, censor_col="censored"):
        self.data = data[[id_col, period_col, treatment_col, outcome_col, eligible_col, censor_col]].copy()
        self.data.columns = ['id', 'period', 'treatment', 'outcome', 'eligible', 'censored']
        return self

def set_censor_weight_model(trial, censor_event, numerator, denominator, pool_models, save_path):
    trial.censor_weights = {
        'censor_event': censor_event,
        'numerator_formula': f"1 - {censor_event} ~ {numerator}",
        'denominator_formula': f"1 - {censor_event} ~ {denominator}",
        'pool_models': pool_models,
        'model_fitter': 'logit',
        'save_path': save_path
    }
    return trial

def set_switch_weight_model(trial, numerator, denominator, save_path):
    trial.switch_weights = {
        'numerator_formula': f"treatment ~ {numerator}",
        'denominator_formula': f"treatment ~ {denominator}",
        'save_path': save_path
    }
    return trial

# Directory Setup
trial_pp_dir = "trial_pp"
os.makedirs(trial_pp_dir, exist_ok=True)
trial_pp_subdir = os.path.join(trial_pp_dir, "switch_models")
os.makedirs(trial_pp_subdir, exist_ok=True)

# Load Data and Initialize trial_pp
data_censored = pd.read_csv("data_censored.csv")  # Ensure this matches your file path
trial_pp = TrialSequence(estimand="PP")
trial_pp = trial_pp.set_data(
    data=data_censored,
    id_col="id",
    period_col="period",
    treatment_col="treatment",
    outcome_col="outcome",
    eligible_col="eligible",
    censor_col="censored"
)

# Set Weight Models
trial_pp = set_censor_weight_model(
    trial_pp,
    censor_event="censored",
    numerator="x2",
    denominator="x2 + x1",
    pool_models="none",
    save_path=trial_pp_subdir
)
trial_pp = set_switch_weight_model(
    trial_pp,
    numerator="age",
    denominator="age + x1 + x3",
    save_path=trial_pp_subdir
)

# Calculate Weights (Fixed)
def calculate_weights(trial):
    # Merge with all necessary columns from data_censored
    data = trial.data.merge(
        data_censored[['id', 'period', 'x1', 'x2', 'x3', 'age']],
        on=['id', 'period'],
        how='left'
    )
    print("Columns after merge:", data.columns.tolist())  # Debug: Check available columns
    
    os.makedirs(trial.censor_weights['save_path'], exist_ok=True)
    
    # Add lagged treatment
    data['prev_treatment'] = data.groupby('id')['treatment'].shift(1)
    print("Columns after adding prev_treatment:", data.columns.tolist())  # Debug
    
    # Filter and drop NA for required columns
    required_cols = ['censored', 'x2', 'x1', 'prev_treatment', 'treatment', 'age', 'x3']
    data_den = data[data['period'] > 0].dropna(subset=required_cols)
    print("Rows in data_den:", len(data_den))  # Debug: Check row count
    
    # Censoring Weights
    for prev_trt in [0, 1]:
        subset = data_den[data_den['prev_treatment'] == prev_trt].copy()
        subset['not_censored'] = 1 - subset['censored']
        # Numerator
        num_model = logit("not_censored ~ x2", data=subset).fit(disp=0)
        trial.censor_weights[f'n{prev_trt}'] = num_model
        num_model.save(os.path.join(trial.censor_weights['save_path'], f"model_n{prev_trt}.pkl"))
        # Denominator
        den_model = logit("not_censored ~ x2 + x1", data=subset).fit(disp=0)
        trial.censor_weights[f'd{prev_trt}'] = den_model
        den_model.save(os.path.join(trial.censor_weights['save_path'], f"model_d{prev_trt}.pkl"))
    
    # Switching Weights
    for prev_trt in [0, 1]:
        subset = data_den[data_den['prev_treatment'] == prev_trt]
        # Numerator
        num_model = logit("treatment ~ age", data=subset).fit(disp=0)
        trial.switch_weights[f'n{prev_trt}'] = num_model
        num_model.save(os.path.join(trial.switch_weights['save_path'], f"model_switch_n{prev_trt}.pkl"))
        # Denominator
        den_model = logit("treatment ~ age + x1 + x3", data=subset).fit(disp=0)
        trial.switch_weights[f'd{prev_trt}'] = den_model
        den_model.save(os.path.join(trial.switch_weights['save_path'], f"model_switch_d{prev_trt}.pkl"))
    
    data['wt'] = 1.0
    trial.data = data
    return trial

# Show Weight Models (Adjusted for Expected Output)
def show_weight_models(trial):
    print("Weight Models for Informative Censoring")
    print("---------------------------------------")
    
    for prev_trt in [0, 1]:
        key = f'n{prev_trt}'
        if key in trial.censor_weights:
            print(f"\n[[{key}]]")
            print(f"Model: P(censor_event = 0 | X, previous treatment = {prev_trt}) for numerator")
            print("\n term        estimate   std.error statistic p.value")
            params = trial.censor_weights[key].params
            std_err = trial.censor_weights[key].bse
            z_stats = trial.censor_weights[key].tvalues
            p_vals = trial.censor_weights[key].pvalues
            for term in params.index:
                term_display = "(Intercept)" if term == "Intercept" else term
                print(f" {term_display:<12} {params[term]:>9.7f} {std_err[term]:>9.7f} {z_stats[term]:>9.7f} {p_vals[term]:.6e}")
            print(f"\n null.deviance df.null logLik    AIC      BIC      deviance df.residual nobs")
            print(f" {trial.censor_weights[key].llnull * -2:>9.4f}      {trial.censor_weights[key].df_resid + trial.censor_weights[key].df_model:<5.0f}     {trial.censor_weights[key].llf:>9.4f} {trial.censor_weights[key].aic:>7.4f} {trial.censor_weights[key].bic:>7.4f} {trial.censor_weights[key].llf * -2:>7.4f} {trial.censor_weights[key].df_resid:<5.0f}         {trial.censor_weights[key].nobs:<5.0f}")
            print(f"\n path")
            print(f" {os.path.join(trial.censor_weights['save_path'], f'model_n{prev_trt}.pkl')}")
    
    for prev_trt in [0, 1]:
        key = f'd{prev_trt}'
        if key in trial.censor_weights:
            print(f"\n[[{key}]]")
            print(f"Model: P(censor_event = 0 | X, previous treatment = {prev_trt}) for denominator")
            print("\n term        estimate   std.error statistic p.value")
            params = trial.censor_weights[key].params
            std_err = trial.censor_weights[key].bse
            z_stats = trial.censor_weights[key].tvalues
            p_vals = trial.censor_weights[key].pvalues
            for term in params.index:
                term_display = "(Intercept)" if term == "Intercept" else term
                print(f" {term_display:<12} {params[term]:>9.7f} {std_err[term]:>9.7f} {z_stats[term]:>9.7f} {p_vals[term]:.6e}")
            print(f"\n null.deviance df.null logLik    AIC      BIC      deviance df.residual nobs")
            print(f" {trial.censor_weights[key].llnull * -2:>9.4f}      {trial.censor_weights[key].df_resid + trial.censor_weights[key].df_model:<5.0f}     {trial.censor_weights[key].llf:>9.4f} {trial.censor_weights[key].aic:>7.4f} {trial.censor_weights[key].bic:>7.4f} {trial.censor_weights[key].llf * -2:>7.4f} {trial.censor_weights[key].df_resid:<5.0f}         {trial.censor_weights[key].nobs:<5.0f}")
            print(f"\n path")
            print(f" {os.path.join(trial.censor_weights['save_path'], f'model_d{prev_trt}.pkl')}")

    print("\nWeight Models for Treatment Switching")
    print("-------------------------------------")
    
    for prev_trt in [1, 0]:
        key = f'n{prev_trt}'
        if key in trial.switch_weights:
            print(f"\n[[{key}]]")
            print(f"Model: P(treatment = 1 | previous treatment = {prev_trt}) for numerator")
            print("\n term        estimate    std.error  statistic p.value")
            params = trial.switch_weights[key].params
            std_err = trial.switch_weights[key].bse
            z_stats = trial.switch_weights[key].tvalues
            p_vals = trial.switch_weights[key].pvalues
            for term in params.index:
                term_display = "(Intercept)" if term == "Intercept" else term
                print(f" {term_display:<12} {params[term]:>9.7f} {std_err[term]:>9.7f} {z_stats[term]:>9.7f} {p_vals[term]:.6e}")
            print(f"\n null.deviance df.null logLik    AIC      BIC      deviance df.residual nobs")
            print(f" {trial.switch_weights[key].llnull * -2:>9.4f}      {trial.switch_weights[key].df_resid + trial.switch_weights[key].df_model:<5.0f}     {trial.switch_weights[key].llf:>9.4f} {trial.switch_weights[key].aic:>7.4f} {trial.switch_weights[key].bic:>7.4f} {trial.switch_weights[key].llf * -2:>7.4f} {trial.switch_weights[key].df_resid:<5.0f}         {trial.switch_weights[key].nobs:<5.0f}")
            print(f"\n path")
            print(f" {os.path.join(trial.switch_weights['save_path'], f'model_switch_n{prev_trt}.pkl')}")

    for prev_trt in [1, 0]:
        key = f'd{prev_trt}'
        if key in trial.switch_weights:
            print(f"\n[[{key}]]")
            print(f"Model: P(treatment = 1 | previous treatment = {prev_trt}) for denominator")
            print("\n term        estimate    std.error  statistic p.value")
            params = trial.switch_weights[key].params
            std_err = trial.switch_weights[key].bse
            z_stats = trial.switch_weights[key].tvalues
            p_vals = trial.switch_weights[key].pvalues
            for term in params.index:
                term_display = "(Intercept)" if term == "Intercept" else term
                print(f" {term_display:<12} {params[term]:>9.7f} {std_err[term]:>9.7f} {z_stats[term]:>9.7f} {p_vals[term]:.6e}")
            print(f"\n null.deviance df.null logLik    AIC      BIC      deviance df.residual nobs")
            print(f" {trial.switch_weights[key].llnull * -2:>9.4f}      {trial.switch_weights[key].df_resid + trial.switch_weights[key].df_model:<5.0f}     {trial.switch_weights[key].llf:>9.4f} {trial.switch_weights[key].aic:>7.4f} {trial.switch_weights[key].bic:>7.4f} {trial.switch_weights[key].llf * -2:>7.4f} {trial.switch_weights[key].df_resid:<5.0f}         {trial.switch_weights[key].nobs:<5.0f}")
            print(f"\n path")
            print(f" {os.path.join(trial.switch_weights['save_path'], f'model_switch_d{prev_trt}.pkl')}")

# Run
trial_pp = calculate_weights(trial_pp)
show_weight_models(trial_pp)

Columns after merge: ['id', 'period', 'treatment', 'outcome', 'eligible', 'censored', 'x1', 'x2', 'x3', 'age']
Columns after adding prev_treatment: ['id', 'period', 'treatment', 'outcome', 'eligible', 'censored', 'x1', 'x2', 'x3', 'age', 'prev_treatment']
Rows in data_den: 636
Weight Models for Informative Censoring
---------------------------------------

[[n0]]
Model: P(censor_event = 0 | X, previous treatment = 0) for numerator

 term        estimate   std.error statistic p.value
 (Intercept)  3.0648508 0.2790704 10.9823587 4.646314e-28
 x2           -0.6378559 0.2543467 -2.5078203 1.214784e-02

 null.deviance df.null logLik    AIC      BIC      deviance df.residual nobs
  134.6812      336        -64.1298 132.2595 139.8997 128.2595 335           337  

 path
 trial_pp\switch_models\model_n0.pkl

[[n1]]
Model: P(censor_event = 0 | X, previous treatment = 1) for numerator

 term        estimate   std.error statistic p.value
 (Intercept)  3.0115791 0.2873389 10.4809306 1.056985e-25
 x